## Проверка содержимого skin-emails.csv и актуальной информации по аннтотации

Только для датасетов с кожей и без возраста

In [2]:
import pandas as pd
import numpy as np
import pymongo as pm
import sys
sys.path.append('../..')
import data

In [3]:
annot = pd.read_pickle('../../data/geo-annotation/samples.annot.res6.pickle')
annot.head()

,accession,norm,series,platform,age,tissue_id,preprocessed
0,GSM305072,1,GSE12103,GPL6848,NaN,BTO:0000089,NaN
1,GSM305305,1,GSE12103,GPL6848,NaN,BTO:0000089,NaN
2,GSM305310,1,GSE12103,GPL6848,NaN,BTO:0000089,NaN
3,GSM1422841,1,GSE58949,GPL18883,NaN,NaN,NaN
4,GSM1422846,1,GSE58949,GPL18883,NaN,NaN,NaN


In [4]:
polina_series = pd.read_csv('/Users/nikita/Data/skin-emails.csv')
polina_series.head()

,series,tissue_id,email
0,GSE10123,skin fibroblast (BTO:0001255),scaffidp@mail.nih.gov
1,GSE10423,skin (BTO:0001253),davewong@stanford.edu
2,GSE10432,dermis (BTO:0000294),dthiboutot@psu.edu
3,GSE10433,skin (BTO:0001253),dthiboutot@psu.edu
4,GSE10434,dermis (BTO:0000294),dthiboutot@psu.edu


In [5]:
polina_series.tissue_id.value_counts()

epidermis (BTO:0000404)           144
skin fibroblast (BTO:0001255)     104
skin (BTO:0001253)                 78
skin cancer cell (BTO:0001286)     78
dermis (BTO:0000294)               31
dtype: int64

In [6]:
series_tissues = (
    annot
    .dropna(subset=['tissue_id'])
    .groupby('series')
    .agg({'tissue_id': lambda t: list(set(t))})
    .assign(l=lambda d: d.tissue_id.map(len))
    .reset_index()
)
series_tissues.head()

,series,tissue_id,l
0,GSE10,[BTO:0001177],1
1,GSE10000,[BTO:0000135],1
2,GSE10001,[BTO:0000759],1
3,GSE10003,[BTO:0000762],1
4,GSE10005,[BTO:0000282],1


In [7]:
polina_series_annot = (
    polina_series
    .rename(columns={'tissue_id': 'tissue_name'})
    .merge(series_tissues.rename(columns={'tissue_id': 'tissue_id_annot'}), on='series', suffixes=('', '_annot'))
    .assign(tissue_id=lambda d: d.tissue_name.map(lambda n: n.split('(')[1].rstrip(')')))
)

polina_series_annot.head()

,series,tissue_name,email,tissue_id_annot,l,tissue_id
0,GSE10123,skin fibroblast (BTO:0001255),scaffidp@mail.nih.gov,[BTO:0001619],1,BTO:0001255
1,GSE10423,skin (BTO:0001253),davewong@stanford.edu,[BTO:0001253],1,BTO:0001253
2,GSE10432,dermis (BTO:0000294),dthiboutot@psu.edu,[BTO:0004613],1,BTO:0000294
3,GSE10434,dermis (BTO:0000294),dthiboutot@psu.edu,[BTO:0004613],1,BTO:0000294
4,GSE10696,epidermis (BTO:0000404),emily.wang@vanderbilt.edu,[BTO:0000404],1,BTO:0000404


In [8]:
polina_series_annot.query("l > 1")

,series,tissue_name,email,tissue_id_annot,l,tissue_id
24,GSE13365,skin fibroblast (BTO:0001255),yamanaka@cira.kyoto-u.ac.jp,"[BTO:0004419, BTO:0004725]",2,BTO:0001255
119,GSE26431,skin fibroblast (BTO:0001255),yamanaka@cira.kyoto-u.ac.jp,"[BTO:0004419, BTO:0004725]",2,BTO:0001255
192,GSE35554,skin fibroblast (BTO:0001255),abara@cfri.ca,"[BTO:0000975, BTO:0001255, BTO:0000759]",3,BTO:0001255
305,GSE52276,epidermis (BTO:0000404),mori9085@hotmail.com,"[BTO:0000759, BTO:0000552, BTO:0001470]",3,BTO:0000404


## Проверяем корректность проставленных tissue_name и tisue_id <-> tissue_id_annot

In [9]:
tissue_ontology = data.tissue_ontology()

Read obo graph
Name: 
Type: DiGraph
Number of nodes: 5659
Number of edges: 6430
Average in degree:   1.1362
Average out degree:   1.1362


In [10]:
# Где не все ткани принадлежат tissue_id
(
    polina_series_annot
    .assign(has_path=lambda d: (
            d
            .tissue_id
            .combine(d.tissue_id_annot, func=(
                    lambda t, tas: all(ta == t or tissue_ontology.has_path(t, ta) for ta in tas)
            ))
        ))
    .query('has_path == False')
)

,series,tissue_name,email,tissue_id_annot,l,tissue_id,has_path
24,GSE13365,skin fibroblast (BTO:0001255),yamanaka@cira.kyoto-u.ac.jp,"[BTO:0004419, BTO:0004725]",2,BTO:0001255,False
119,GSE26431,skin fibroblast (BTO:0001255),yamanaka@cira.kyoto-u.ac.jp,"[BTO:0004419, BTO:0004725]",2,BTO:0001255,False
192,GSE35554,skin fibroblast (BTO:0001255),abara@cfri.ca,"[BTO:0000975, BTO:0001255, BTO:0000759]",3,BTO:0001255,False
305,GSE52276,epidermis (BTO:0000404),mori9085@hotmail.com,"[BTO:0000759, BTO:0000552, BTO:0001470]",3,BTO:0000404,False


## Проверяем высланы ли все датасеты

т.е. чтобы в выборке *annot* с тканью (skin и уровнем ниже) и **без** возраста все датасеты совпали с *polina_series.series*

In [11]:
polina_series.series.unique().shape

(435,)

In [52]:
from collections import defaultdict

def get_parents(ontology, parent_nodes):
    mapping = defaultdict(list)
    for node in ontology.meta.keys():
        for parent_node in parent_nodes:
            if ontology.has_path(parent_node, node):
                mapping[node].append(parent_node)
    return lambda t: mapping.get(t, np.nan)

skin_descendants = tissue_ontology.descendants('BTO:0001253')
# map_f = tissue_ontology.to_parent_f(['BTO:0001253'] + skin_descendants)
map_f_desc = get_parents(tissue_ontology, skin_descendants)
map_f_skin = get_parents(tissue_ontology, ['BTO:0001253'])

In [50]:
_annot_desc = (
    annot[annot.age.isnull()]
    .assign(parent_tissue_id=lambda d: d.tissue_id.map(map_f_desc))
)

_annot_skin = (
    annot[annot.age.isnull()]
    .assign(parent_tissue_id=lambda d: d.tissue_id.map(map_f_skin))
)

annot_skin_noage = (
    _annot_desc.fillna(_annot_skin)
    .dropna(subset=['parent_tissue_id'])
)


In [51]:
annot_skin_noage.head()

,accession,norm,series,platform,age,tissue_id,preprocessed,parent_tissue_id
28,GSM1434757,1,GSE59307,GPL570,NaN,BTO:0001253,1,[BTO:0001253]
54,GSM1434759,1,GSE59307,GPL570,NaN,BTO:0001253,1,[BTO:0001253]
55,GSM1434763,1,GSE59307,GPL570,NaN,BTO:0001253,1,[BTO:0001253]
57,GSM1434762,1,GSE59307,GPL570,NaN,BTO:0001253,1,[BTO:0001253]
58,GSM1434761,1,GSE59307,GPL570,NaN,BTO:0001253,1,[BTO:0001253]


In [20]:
def first(xs):
    return next(iter(xs))

Выгружаем все поля *email* для серий

In [21]:
series_emails = (pd.DataFrame(list(
    pm.MongoClient().scraper_series_email.series.find({}, {'_id':0 , 'accession': 1, 'email': 1})
))
                .rename(columns={'accession': 'series'})
                )
series_emails.head()

,series,email
0,GSE2,ykagami@brain.riken.go.jp
1,GSE1,mbittner@nhgri.nih.gov
2,GSE10,dror_sharon@meei.harvard.edu
3,GSE452,deans@itsa.ucsf.edu
4,E-ERAD-241,NaN


In [35]:
from itertools import chain
def list_set(iterables):
    return list(set(iterables))

def merge_lists(iterable_of_iterables):
    return list(set(chain(*(list(iterable_of_iterables)))))

Приводим в вид с подписанными болезнями и мерджим с датасетом *skin-emails*

In [ ]:
def cell():
    return (
            annot_skin_noage
            .dropna(subset=['parent_tissue_id'])
            .groupby('series', as_index=False)
            .agg({'parent_tissue_id': merge_lists,
                  'tissue_id': list_set})
            .assign(l=lambda d: d.tissue_id.map(len))
            .query('l == 1')
            .assign(tissue_id=lambda d: d.tissue_id.map(first))
            .assign(tissue_name=lambda d: d.tissue_id.map(tissue_ontology.name),
                    parent_tissue_name=lambda d: d.parent_tissue_id.map(
                        lambda ts: ','.join(map(tissue_ontology.name, ts))
                   ))
                    
            .drop('l', axis=1)
            .merge(polina_series, on='series', suffixes=('', '_polina'), how='left')
            .rename(columns={'tissue_id_polina': 'was_in_first_dataset'})
            .assign(was_in_first_dataset=lambda d: d.was_in_first_dataset.map(lambda x: 1 if pd.notnull(x) else 0))
            .assign(parent_tissue_name=lambda d: d.parent_tissue_id.map(
                            lambda ts: ','.join(map(lambda t: '{} ({})'.format(tissue_ontology.name(t), t), ts))
                        )
                   )
            .assign(tissue_name=lambda d: d.tissue_name.combine(d.tissue_id, '{} ({})'.format))
            .merge(series_emails, how='left')

        )

series_tissues_email = cell()

Видно, что где *email* == *NaN*, то и не отправляли

In [97]:
series_tissues_email.head()

,series,tissue_id,parent_tissue_id,parent_tissue_name,tissue_name,was_in_first_dataset,email
0,GSE10085,BTO:0001858,[BTO:0000294],dermis (BTO:0000294),dermal papilla (BTO:0001858),0,NaN
1,GSE10098,BTO:0005055,[BTO:0000404],epidermis (BTO:0000404),scale (BTO:0005055),0,NaN
2,GSE10118,BTO:0001858,[BTO:0000294],dermis (BTO:0000294),dermal papilla (BTO:0001858),0,NaN
3,GSE10123,BTO:0001619,[BTO:0001255],skin fibroblast (BTO:0001255),skin fibroblast cell line (BTO:0001619),1,scaffidp@mail.nih.gov
4,GSE10423,BTO:0001253,[BTO:0001253],skin (BTO:0001253),skin (BTO:0001253),1,davewong@stanford.edu


In [ ]:
! mkdir -p ../../data/geo-annotation/intermediate/skin

## Запись результатов

In [98]:
(
    series_tissues_email
    [series_tissues_email.email.isnull()]
    [['series', 'tissue_name',  'parent_tissue_name']]
    .to_csv('../../data/geo-annotation/intermediate/skin/skin-emails.csv', index=False)
)

## Сверяем размеры датасетов
* Сколько всего датасетов с кожей и без возраста
* Сколько всего датасетов с указанным email
* Сколько всего датасетов, в высланной таблице полине

Видно что из ~ 1000 только 430 имеют *email*,  однако почемуто 5 датасетов пропало в новой выдаче (*series_tissues_email*)

In [96]:
series_tissues_email.shape, series_tissues_email.dropna(subset=['email']).shape, polina_series.shape

((998, 7), (430, 7), (435, 3))

Сравним что изменилось

In [70]:
def cell():
    return (
        series_tissues_email
        .dropna(subset=['email'])
        .merge(polina_series, how='outer', on='series', suffixes=('_new', '_old'))
    )
new_vs_old = cell()
new_vs_old.head()

,series,tissue_id_new,parent_tissue_id,parent_tissue_name,tissue_name,was_in_first_dataset,email_new,tissue_id_old,email_old
0,GSE10123,BTO:0001619,[BTO:0001255],skin fibroblast (BTO:0001255),skin fibroblast cell line (BTO:0001619),1,scaffidp@mail.nih.gov,skin fibroblast (BTO:0001255),scaffidp@mail.nih.gov
1,GSE10423,BTO:0001253,[BTO:0001253],skin (BTO:0001253),skin (BTO:0001253),1,davewong@stanford.edu,skin (BTO:0001253),davewong@stanford.edu
2,GSE10432,BTO:0004613,[BTO:0000294],dermis (BTO:0000294),sebocyte (BTO:0004613),1,dthiboutot@psu.edu,dermis (BTO:0000294),dthiboutot@psu.edu
3,GSE10434,BTO:0004613,[BTO:0000294],dermis (BTO:0000294),sebocyte (BTO:0004613),1,dthiboutot@psu.edu,dermis (BTO:0000294),dthiboutot@psu.edu
4,GSE10696,BTO:0000404,[BTO:0000404],epidermis (BTO:0000404),epidermis (BTO:0000404),1,emily.wang@vanderbilt.edu,epidermis (BTO:0000404),emily.wang@vanderbilt.edu


In [71]:
new_vs_old[new_vs_old.tissue_id_new.isnull()]

,series,tissue_id_new,parent_tissue_id,parent_tissue_name,tissue_name,was_in_first_dataset,email_new,tissue_id_old,email_old
430,GSE10433,NaN,NaN,NaN,NaN,NaN,NaN,skin (BTO:0001253),dthiboutot@psu.edu
431,GSE28384,NaN,NaN,NaN,NaN,NaN,NaN,dermis (BTO:0000294),weidner@molgen.mpg.de
432,GSE33092,NaN,NaN,NaN,NaN,NaN,NaN,epidermis (BTO:0000404),kqu@stanford.edu
433,GSE40122,NaN,NaN,NaN,NaN,NaN,NaN,epidermis (BTO:0000404),kqu@stanford.edu
434,GSE52276,NaN,NaN,NaN,NaN,NaN,NaN,epidermis (BTO:0000404),mori9085@hotmail.com


In [72]:
new_vs_old[new_vs_old.tissue_id_old.isnull()]

,series,tissue_id_new,parent_tissue_id,parent_tissue_name,tissue_name,was_in_first_dataset,email_new,tissue_id_old,email_old


In [73]:
new_vs_old[new_vs_old.email_new.isnull()]

,series,tissue_id_new,parent_tissue_id,parent_tissue_name,tissue_name,was_in_first_dataset,email_new,tissue_id_old,email_old
430,GSE10433,NaN,NaN,NaN,NaN,NaN,NaN,skin (BTO:0001253),dthiboutot@psu.edu
431,GSE28384,NaN,NaN,NaN,NaN,NaN,NaN,dermis (BTO:0000294),weidner@molgen.mpg.de
432,GSE33092,NaN,NaN,NaN,NaN,NaN,NaN,epidermis (BTO:0000404),kqu@stanford.edu
433,GSE40122,NaN,NaN,NaN,NaN,NaN,NaN,epidermis (BTO:0000404),kqu@stanford.edu
434,GSE52276,NaN,NaN,NaN,NaN,NaN,NaN,epidermis (BTO:0000404),mori9085@hotmail.com


In [74]:
new_vs_old[new_vs_old.email_old.isnull()]

,series,tissue_id_new,parent_tissue_id,parent_tissue_name,tissue_name,was_in_first_dataset,email_new,tissue_id_old,email_old


In [84]:
new_vs_old.query('parent_tissue_name != tissue_id_old') \
[['series', 'tissue_name', 'parent_tissue_name', 'tissue_id_old']][:20]


,series,tissue_name,parent_tissue_name,tissue_id_old
7,GSE11550,hs-294t cell (BTO:0003467),"epidermis (BTO:0000404),skin cancer cell (BTO:...",skin cancer cell (BTO:0001286)
8,GSE11551,hs-294t cell (BTO:0003467),"epidermis (BTO:0000404),skin cancer cell (BTO:...",skin cancer cell (BTO:0001286)
9,GSE11552,hs-294t cell (BTO:0003467),"epidermis (BTO:0000404),skin cancer cell (BTO:...",skin cancer cell (BTO:0001286)
13,GSE12096,melanoma cell (BTO:0000848),"epidermis (BTO:0000404),skin cancer cell (BTO:...",skin cancer cell (BTO:0001286)
22,GSE13334,dermal fibroblast (BTO:0004419),"skin fibroblast (BTO:0001255),dermis (BTO:0000...",skin fibroblast (BTO:0001255)
24,GSE13365,dermal fibroblast (BTO:0004419),"skin fibroblast (BTO:0001255),dermis (BTO:0000...",skin fibroblast (BTO:0001255)
34,GSE15138,melanoma cell line (BTO:0000849),"epidermis (BTO:0000404),skin cancer cell (BTO:...",skin cancer cell (BTO:0001286)
36,GSE15281,melanoma cell line (BTO:0000849),"epidermis (BTO:0000404),skin cancer cell (BTO:...",skin cancer cell (BTO:0001286)
37,GSE15420,melanoma cell line (BTO:0000849),"epidermis (BTO:0000404),skin cancer cell (BTO:...",skin cancer cell (BTO:0001286)
40,GSE1562,melanoma cell line (BTO:0000849),"epidermis (BTO:0000404),skin cancer cell (BTO:...",skin cancer cell (BTO:0001286)


In [90]:
new_vs_old[new_vs_old.email_new.isnull()].query('tissue_id_old not in parent_tissue_name') \
[['series', 'tissue_name', 'parent_tissue_name', 'tissue_id_old']][:60]


,series,tissue_name,parent_tissue_name,tissue_id_old
430,GSE10433,NaN,NaN,skin (BTO:0001253)
431,GSE28384,NaN,NaN,dermis (BTO:0000294)
432,GSE33092,NaN,NaN,epidermis (BTO:0000404)
433,GSE40122,NaN,NaN,epidermis (BTO:0000404)
434,GSE52276,NaN,NaN,epidermis (BTO:0000404)


In [91]:
new_vs_old.query('tissue_id_old not in parent_tissue_name') \
[['series', 'tissue_name', 'parent_tissue_name', 'tissue_id_old']][:60]


,series,tissue_name,parent_tissue_name,tissue_id_old
